Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

In [62]:
%load_ext sql
%sql sqlite:///chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: None@chinook.db'

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [63]:
%%sql
Pragma foreign_key=ON;
DROP TABLE if exists PlayersArchive;
DROP TABLE if exists Players;
DROP TABLE if exists Position;
DROP TABLE if exists Teams;
DROP TABLE if exists Coaches;

CREATE TABLE Coaches (ID integer(0,0) NOT NULL Primary key, name varchar(0,0)); 

CREATE TABLE Teams (ID integer(0,0) NOT NULL Primary key,
                    name varchar(0,0) NOT NULL,
                    location varchar(0,0),
                    coach_id integer(0,0) NOT NULL,
                    FOREIGN KEY (coach_id) REFERENCES Coaches(ID));

CREATE TABLE  Position (ID integer(0,0) NOT NULL Primary key, name varchar(0,0)); 

CREATE TABLE  Players (ID integer(0,0) NOT NULL Primary key,
                       name varchar(0,0) NOT NULL, 
                       age intenger(0,0),
                       team_id intenger(0,0),
                       position_id intenger(0,0),
                       start_date date,
                       FOREIGN KEY (team_id) REFERENCES Teams(ID),
                       FOREIGN KEY (position_id) REFERENCES Position(ID));

CREATE TABLE PlayersArchive (player_ID integer(0,0) NOT NULL, 
                            team_id integer(0,0) NOT NULL,
                            start_date date(0,0) NOT NULL,
                            end_date date(0,0) NOT NULL,
                            FOREIGN KEY (player_ID) REFERENCES Players(ID)
                            FOREIGN KEY (team_id) REFERENCES Teams(ID));

Done.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.
8 rows affected.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [64]:
%%sql
INSERT INTO Position VALUES (1,'Вратарь');
INSERT INTO Position VALUES (2,'Защитник');
INSERT INTO Position VALUES (3,'Полузащитник');
INSERT INTO Position VALUES (4,'Нападающий');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [65]:
%%sql 
select *
from Position 

Done.


ID,name
1,Вратарь
2,Защитник
3,Полузащитник
4,Нападающий


# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [66]:
import sqlite3;
db=sqlite3.connect('chinook.db')
def check_coach(coach_name):
    cur=db.cursor()
    cur.execute('''select id from coaches where name =?''',(coach_name,))
    try:
        coach_id=cur.fetchone()[0]
        print('Тренер',coach_name, 'уже записан')
        return coach_id
    except TypeError:
        return -1
    
def update_coach(coach_name):
    coaches_check = check_coach(coach_name)
    if coaches_check != -1:
        return -1
    cur=db.cursor()
    cur.execute( '''SELECT COALESCE(MAX(ID)+1, 1) FROM coaches''')
    ID_coach=cur.fetchone()[0]
    cur.execute('''INSERT INTO coaches VALUES(?,?)''', (ID_coach,coach_name))
    db.commit()

In [72]:
update_coach('Михаил');
update_coach('Иван');
update_coach('Алексей');
update_coach('Сергей');
update_coach('Петр');
update_coach('Михаил'); #для проверки 

Тренер Михаил уже записан


In [73]:
%sql select * from coaches

Done.


ID,name
1,Михаил
2,Иван
3,Алексей
4,Сергей
5,Петр


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [74]:
def check_team(team_name):
    cur=db.cursor()
    cur.execute('''select id from teams where name =?''',(team_name,))
    try:
        team_id=cur.fetchone()[0]
        print('Команда',team_name, 'уже существует')
        return team_id
    except TypeError:
        return -1
    
def ID_Coach(coach_name):
    cur=db.cursor()
    cur.execute('''select id from coaches where name =?''',(coach_name,))
    try:
        coach_id=cur.fetchone()[0]
        return coach_id
    except TypeError:
        print('Тренера',coach_name, 'не существует')
        return -1
    
def check_coach(Coach_ID):
    cur=db.cursor()
    cur.execute('''select id from teams where coach_id =?''',(Coach_ID,))
    try:
        team_id=cur.fetchone()[0]
        return team_id
    except TypeError:
        return -1        
    
def update_Team(team_name, location,coach_name):
    team_check = check_team(team_name)
    if team_check != -1:
        return -1
    cur=db.cursor()
    cur.execute( '''SELECT COALESCE(MAX(ID)+1, 1) FROM teams''')
    Team_ID=cur.fetchone()[0]
    Coach_ID = ID_Coach(coach_name) 
    if Coach_ID == -1:
        return -1
    Coach_check = check_coach(Coach_ID) #проверяем, вдруг такой тренер уже кого-то тренерует (не знаю, нужно или нет)
    if  Coach_check != -1:
        print('Тренер',coach_name, 'уже тренерует другую команду')
        return -1
    cur.execute('''INSERT INTO teams VALUES(?,?,?,?)''', (Team_ID,team_name,location,Coach_ID))
    db.commit()

In [75]:
update_Team('Ягодки','Москва','Алексей');
update_Team('Цветочки','Лондон','Михаил');
update_Team('Бабочки','Санкт-Петербург','Иван');
update_Team('Конфетки','Санкт-Петербург','Андрей');
update_Team('Снежинки','Москва','Сергей');
update_Team('Цветочки','Уфа','Артем');
update_Team('Льдинки','Москва','Сергей');

Тренера Андрей не существует
Команда Цветочки уже существует
Тренер Сергей уже тренерует другую команду


In [76]:
%sql select * from teams

Done.


ID,name,location,coach_id
1,Ягодки,Москва,3
2,Цветочки,Лондон,1
3,Бабочки,Санкт-Петербург,2
4,Снежинки,Москва,4


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [77]:
def ID_team(team_name):
    cur=db.cursor()
    cur.execute('''select id from teams where name =?''',(team_name,))
    try:
        team_id=cur.fetchone()[0]
        return team_id
    except TypeError:
        print('Команды',team_name,'не существует')
        return -1   
    
def ID_coach(coach):
    cur=db.cursor()
    cur.execute('''select id from coaches where name =?''',(coach,))
    try:
        coach_id=cur.fetchone()[0]
        return coach_id
    except TypeError:
        return -1      
    
def update_team(*args):
    cur=db.cursor()
    length=len(args)
    commit_flag=0
    team_name=args[0]
    team_ID = ID_team(team_name) 
    if team_ID == -1:
        return -1
    coach_id=ID_coach(args[1]) 
    if coach_id == -1:
        location=args[1]
        commit_flag=1
    else:
        coach=args[1]
        commit_flag=2
    if length == 3:
        location=args[2]
        commit_flag=3
    if commit_flag==1:
         cur.execute('''update teams set location=? where name=?''', (location,team_name))
    elif commit_flag==2:
         cur.execute('''update teams set coach_id=? where name=?''', (coach_id,team_name))
    else:
         cur.execute('''update teams set location=?, coach_id=? where name=?''', (location,coach_id,team_name))
    db.commit()
    

In [78]:
update_team('Цветочки','Нью-Йорк')
update_team('Цветочки','Петр')

In [79]:
%sql select * from teams

Done.


ID,name,location,coach_id
1,Ягодки,Москва,3
2,Цветочки,Нью-Йорк,5
3,Бабочки,Санкт-Петербург,2
4,Снежинки,Москва,4


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [57]:
import datetime

def check_player(player_name):
    cur=db.cursor()
    cur.execute('''select id from Players where name =?''',(player_name,))
    try:
        player_id=cur.fetchone()[0]
        print('Игрок',player_name, 'уже существует')
        return player_id
    except TypeError:
        return -1

def ID_position(position):
    cur=db.cursor()
    cur.execute('''select id from position where name =?''',(position,))
    try:
        position_id=cur.fetchone()[0]
        return position_id
    except TypeError:
        print('Позиции',position,'не существует')
        return -1   
    
def ID_team(team_name):
    cur=db.cursor()
    cur.execute('''select id from teams where name =?''',(team_name,))
    try:
        team_id=cur.fetchone()[0]
        return team_id
    except TypeError:
        print('Команды',team_name,'не существует')
        return -1      
    
def update_player(*args):
    length=len(args)
    player_name=args[0]
    player_check = check_player(player_name)
    if player_check != -1:
        return -1
    cur=db.cursor()
    cur.execute( '''SELECT COALESCE(MAX(ID)+1, 1) FROM Players''')
    player_ID = cur.fetchone()[0]
    age=args[1]
    if age<10 or age>50:
        print(player_name,'неподходящий возраст')
        return -1
    position=args[2]
    position_ID = ID_position(position) 
    if position_ID == -1:
        return -1
    team=args[3]
    team_ID = ID_team(team) 
    if team_ID == -1:
        return -1
    if length==4:
        data= datetime.date.today()
    else:
        data=args[4]
    cur.execute('''INSERT INTO Players VALUES(?,?,?,?,?,?)''', (player_ID,player_name,age, position_ID,team_ID,data))
    db.commit()

In [58]:
update_player('Георгий',15,'Нападающий','Цветочки','2017-03-12');
update_player('Леопольд',20,'Вратарь','Ягодки','2016-02-18');
update_player('Аркадий',9,'Защитник','Снежинки','2015-12-30');
update_player('Евгений',18,'Полузащитник','Бабочки','2017-10-16');
update_player('Георгий',15,'Нападающий','Цветочки','2017-11-20');
update_player('Василий',19,'Нападающий','Цветочки','2015-06-01');
update_player('Виталий',26,'Защитник','Ягодки','2014-08-07');
update_player('Олег',11,'Защитник','Снежинки');
update_player('Антон',34,'Вратарь','Бабочки','2012-11-30');
update_player('Юрий',18,'Защитник','Гусеницы');
update_player('Константин',45,'Судья','Бабочки','2017-03-12');
update_player('Илья',45,'Вратарь','Снежинки');

Аркадий неподходящий возраст
Игрок Георгий уже существует
Команды Гусеницы не существует
Позиции Судья не существует


In [59]:
%sql select * from  Players

Done.


ID,name,age,team_id,position_id,start_date
1,Георгий,15,4,2,2017-03-12
2,Леопольд,20,1,1,2016-02-18
3,Евгений,18,3,3,2017-10-16
4,Василий,19,4,2,2015-06-01
5,Виталий,26,2,1,2014-08-07
6,Олег,11,2,4,2017-05-06
7,Антон,34,1,3,2012-11-30
8,Илья,45,1,4,2017-05-06


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.